In [628]:
#Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [629]:
#GPU Checking
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 3060


In [630]:
# Text Sample
text = "This is a simple example to demonstrate how to predict the next character using LSTM in PyTorch."

In [631]:
# **Taken from the course GitHub**

#set(text): Creates a set of unique characters found in the text. The set function removes any duplicate characters.
#list(set(text)): Converts the set back into a list so that it can be sorted. 
# sorted(list(set(text))): Sorts the list of unique characters. 
chars = sorted(list(set(text)))
#This line creates a dictionary that maps each character to a unique index (integer)."
ix_to_char = {i: ch for i, ch in enumerate(chars)}
#Similar to the previous line, but in reverse. This line creates a dictionary that maps each unique index (integer) back to its corresponding character.
char_to_ix = {ch: i for i, ch in enumerate(chars)} 
chars = sorted(list(set(text)))

In [632]:
# Preparing the dataset
max_length = 70  # Maximum length of input sequences
X = []
y = []
for i in range(len(text) - max_length):
    sequence = text[i:i + max_length]
    label = text[i + max_length]
    X.append([char_to_ix[char] for char in sequence])
    y.append(char_to_ix[label])

X = np.array(X)
y = np.array(y)

In [633]:
# Splitting the dataset 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

# Converting to tensors
X_train = torch.tensor(X_train, dtype=torch.long).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_val = torch.tensor(X_val, dtype=torch.long).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)

In [634]:
#Training Loop + Empty Arrays for storing Training and Validation Results

def TL(n_epochs, optimizer, model, loss_fn, X_train, y_train, X_val, y_val):
  for epoch in range(1, n_epochs + 1):
    #Training Loop
    model.train()
    optimizer.zero_grad()
    train_out = model(X_train)
    loss = loss_fn(train_out, y_train)
    loss.backward()
    optimizer.step()
    
    #Validation Loop
    model.eval()
    with torch.no_grad():
      val_out = model(X_val)
      val_loss = loss_fn(val_out, y_val)
      _, predicted = torch.max(val_out, 1)
      val_accuracy = (predicted == y_val).float().mean()
    

    if epoch == 1 or epoch % 5 == 0: 
      print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Validation Accuracy: {}'.format(epoch, loss.item(), val_loss.item(), val_accuracy.item()))


Question 1 - Part B: LSTM

In [635]:
# LSTM Model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size 
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers= num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])
        return output

In [636]:
hidden_size = 128
model = LSTM(len(chars), hidden_size, len(chars), 2).to(device)
a = 0.0001
epochs = 200
lossFN = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr= a)
start_time = time.time()

TL(
  n_epochs= epochs,
  optimizer= optimizer,
  model= model,
  loss_fn= lossFN,
  X_train= X_train,
  y_train= y_train,
  X_val= X_val,
  y_val= y_val
)

#Getting the time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} Seconds")

Epoch: 1, Training Loss: 3.2488675117492676, Validation Loss: 3.2519102096557617, Validation Accuracy: 0.0
Epoch: 5, Training Loss: 3.248842716217041, Validation Loss: 3.251910448074341, Validation Accuracy: 0.0
Epoch: 10, Training Loss: 3.248811721801758, Validation Loss: 3.251911163330078, Validation Accuracy: 0.0
Epoch: 15, Training Loss: 3.2487804889678955, Validation Loss: 3.251912832260132, Validation Accuracy: 0.0
Epoch: 20, Training Loss: 3.2487494945526123, Validation Loss: 3.251915216445923, Validation Accuracy: 0.0
Epoch: 25, Training Loss: 3.248718738555908, Validation Loss: 3.2519168853759766, Validation Accuracy: 0.0
Epoch: 30, Training Loss: 3.248688220977783, Validation Loss: 3.251918077468872, Validation Accuracy: 0.0
Epoch: 35, Training Loss: 3.2486579418182373, Validation Loss: 3.251919984817505, Validation Accuracy: 0.0
Epoch: 40, Training Loss: 3.248627185821533, Validation Loss: 3.251922607421875, Validation Accuracy: 0.0
Epoch: 45, Training Loss: 3.24859499931335

Epoch: 65, Training Loss: 3.2484729290008545, Validation Loss: 3.251927375793457, Validation Accuracy: 0.0
Epoch: 70, Training Loss: 3.248441219329834, Validation Loss: 3.2519280910491943, Validation Accuracy: 0.0
Epoch: 75, Training Loss: 3.248410701751709, Validation Loss: 3.2519290447235107, Validation Accuracy: 0.0
Epoch: 80, Training Loss: 3.248379945755005, Validation Loss: 3.2519302368164062, Validation Accuracy: 0.0
Epoch: 85, Training Loss: 3.2483506202697754, Validation Loss: 3.2519311904907227, Validation Accuracy: 0.0
Epoch: 90, Training Loss: 3.248319625854492, Validation Loss: 3.251932382583618, Validation Accuracy: 0.0
Epoch: 95, Training Loss: 3.2482879161834717, Validation Loss: 3.2519328594207764, Validation Accuracy: 0.0
Epoch: 100, Training Loss: 3.2482573986053467, Validation Loss: 3.2519352436065674, Validation Accuracy: 0.0
Epoch: 105, Training Loss: 3.2482268810272217, Validation Loss: 3.251936197280884, Validation Accuracy: 0.0
Epoch: 110, Training Loss: 3.2481

In [637]:
# Prediction Fn
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

In [638]:
#Complexity
numel_list = [p.numel() for p in model.parameters()]
print("Model Complexity: {}".format(sum(numel_list)))
#sum(numel_list), numel_list

model = model.to('cpu')

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Model Complexity: 270874
Predicted next character: 'S'
